In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
np.set_printoptions(suppress=True)

In [4]:
import sys
sys.path.insert(0, 'preprocess/')
import vectorizer

import pickle

In [5]:
vec = pickle.load(open('preprocess/SST/sst.p', 'rb'))

In [6]:
import model.Attn_Word as AttnModel
Model = AttnModel.Model

In [7]:
vocab_size = vec.vocab_size
embed_size = vec.word_dim

In [8]:
X, Xt = vec.seq_text['train'], vec.seq_text['test']
y, yt = vec.label['train'], vec.label['test']

In [9]:
ind_1 = [i for i, x in enumerate(X) if len(x) <= 2]
X = [x for i, x in enumerate(X) if i not in ind_1]
y = [x for i, x in enumerate(y) if i not in ind_1]
print(len(ind_1))
ind_1 = [i for i, x in enumerate(Xt) if len(x) <= 2]
Xt = [x for i, x in enumerate(Xt) if i not in ind_1]
yt = [x for i, x in enumerate(yt) if i not in ind_1]

0


In [10]:
def train(std, lreg, kld=False, name='') :
    model = Model(vocab_size, embed_size, 32, dirname='sst', pre_embed=vec.embeddings)
    model.std = std
    model.lreg = lreg
    loss_list = []

    for i in tqdm_notebook(range(7)) :
        if kld :
            if i >= 1 :
                correct=True
            else :
                correct=False
        else : correct = False
        loss, losses = model.train(X, y, correct=correct)
        print(loss)
        loss_list.append(losses)
        o, he = model.evaluate(Xt)
        o = np.array(o)
        rep = classification_report(yt, (o > 0.5))
        print(rep)
        stmt = '%s, %s, %s, %s' % (i, loss, std, lreg)
        dirname = model.save_values(add_name=name+'_test_'+str(std)+'_'+str(lreg), save_model=(not np.isnan(loss)))
        f = open(dirname + '/epoch.txt', 'a')
        f.write(stmt + '\n')
        f.write(rep + '\n')
        f.close()
    
    pickle.dump(loss_list, open(dirname + '/loss_list.p', 'wb'))
    model.loss_list = loss_list
    return model

In [14]:
lreg = [0.005, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2]
std = [0.005, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 2]

In [15]:
model = train(std[0], lreg[0], kld=False, name='test')

Setting Embedding


tensor(1.00000e-03 *
       [[ 1.5029,  1.0030,  3.2585,  ...,  0.0000,  0.0000,  0.0000],
        [-0.7101, -1.6123, -1.2102,  ...,  0.0000,  0.0000,  0.0000],
        [-0.4603,  1.1363,  2.2358,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.9933,  2.0050, -0.6084,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1151, -1.5193, -0.4917,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.0062, -0.3347, -2.0573,  ...,  0.0000,  0.0000,  0.0000]], device='cuda:0')



NameError: name 'ss' is not defined

In [ ]:
for lr in lreg :
    for s in std :
        if lr <= 1 and s <= 1 :
            continue
        print(s, lr)
        model = train(s, lr, kld=True, name='kld')

In [ ]:
def load_model(dirname) :
    model = Model(vocab_size, embed_size, 32, dirname='sst', pre_embed=vec.embeddings)
    model.std = 2
    model.lreg = 1
    model.dirname = dirname
    model.load_values(dirname)
    return model

In [ ]:
dirname_normal = 'outputs/attn_word_sst/ThuJul1213:15:002018_normal/'
model_normal = load_model(dirname_normal)

In [ ]:
dirname_kld = 'outputs/attn_word_sst/FriJul1300:08:552018_kld_test_2_1.5/'
model_kld = load_model(dirname_kld)

In [ ]:
import pickle
loss_list = pickle.load(open('outputs/attn_word_sst/FriJul1300:50:572018_kld_test_2_2/loss_list.p', 'rb'))

Evaluation
==========

In [ ]:
Xtest = Xt
ytest = yt

In [ ]:
def generate_sampled(model) :
    attn_list = []
    sampled_list = []
    for i in range(10) :
        _, attn, sampled = model.evaluate(Xtest, sample=True)
        attn_list.append(attn)
        sampled_list.append(sampled)
        
    pickle.dump([attn_list, sampled_list], open(model.dirname + '/sampled.p', 'wb'))
        
def load_sampled(model) :
    attn_list, sampled_list = pickle.load(open(model.dirname + '/sampled.p', 'rb'))
    attn = list(map(list, zip(*attn_list)))
    attn = [np.array(x) for x in attn]
    attn = [x.mean(0) for x in attn]

    sampled = list(map(list, zip(*sampled_list)))
    sampled = [np.array(x) for x in sampled]
    sampled = [x.mean(0) for x in sampled]

    model.attn = attn
    model.sampled = sampled

In [ ]:
import os
files = os.listdir('outputs/attn_word_sst/')
for f in tqdm_notebook(files) :
    try :
        dirname_kld = 'outputs/attn_word_sst/' + f
        if os.path.isfile(dirname_kld + '/sampled.p') :
            continue
        model_kld = load_model(dirname_kld)
        generate_sampled(model_kld)
    except :
        continue

In [ ]:
# generate_sampled(model_normal)
generate_sampled(model_kld)

In [ ]:
models = {}

for f in files :
    try :
        g = f.split('_')
        std = float(g[-2])
        lreg = float(g[-1])
        dirname_kld = 'outputs/attn_word_sst/' + f
        models[(std, lreg)] = load_model(dirname_kld)
        load_sampled(models[(std, lreg)])
    except Exception as e:
        print(e)
        continue

In [ ]:
len(models)

In [ ]:
def calc_corr(model) :
    a = []
    b = []
    for i in range(len(model.attn)) :
        a += list(model.attn[i])
        b += list(model.sampled[i])
        
    ind = [i for i, (x, y) in enumerate(zip(a, b)) if y == y and x > 0.0]

    a = np.array(a)[ind]
    b = np.array(b)[ind]
    model.spear = spearmanr(a, b)
    model.kendall = kendalltau(a, b)
    model.pear = np.corrcoef(a, b)

In [ ]:
for m in tqdm_notebook(models) :
    calc_corr(models[m])

In [ ]:
scores = np.zeros((len(lreg), len(std)))

In [ ]:
for i, l in enumerate(lreg) :
    for j, s in enumerate(std) :
        try :
            scores[i, j] = models[(s, l)].pear[0, 1]
        except :
            scores[i, j] = models[(1.5, 1.0)].pear[0, 1]

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(scores)
plt.xticks(range(len(std)), std)
plt.yticks(range(len(lreg)), lreg)
for i in range(8) :
    for j in range(8) :
        plt.text(i, j, "{0:.3f}".format(scores[j, i]))

In [ ]:
from scipy.stats import spearmanr, kendalltau
from cycler import cycler
import matplotlib.pyplot as plt

plt.rc('lines', linewidth=2)
line_cycle = cycler('linestyle', ['-', '--'])
color_cycle = cycler('color', ['#1f78b4', '#33a02c'])
marker_cycle = cycler('marker', ['+', '2'])

def plot(attn, sampled, ax, name, marker) :
    a = []
    b = []
    for i in range(len(attn)) :
        a += list(attn[i])
        b += list(sampled[i])
        
    ind = [i for i, (x, y) in enumerate(zip(a, b)) if y == y and x > 0.0]

    a = np.array(a)[ind]
    b = np.array(b)[ind]
    print(spearmanr(a, b))
    print(kendalltau(a, b))
    print(np.corrcoef(a, b))
    fit = np.polyfit(a, b, deg=1)
    r = np.linspace(0, 1, 10)
    ax.plot(r, fit[0] * r + fit[1], label=name + (r' ($\rho$=%.3f'%np.corrcoef(a, b)[0, 1]) + ')')
    ax.scatter(a, b, s=0.8, alpha=0.5, marker=marker)
    ax.set_xlabel('Attention')
    ax.set_ylabel('Expected Normalised KL divergence')

In [ ]:
fig, ax = plt.subplots()
ax.set_prop_cycle(line_cycle + color_cycle) # + marker_cycle)
# plot(model_normal.attn, model_normal.sampled, ax, name='Standard', marker='^')
plot(model_kld.attn, model_kld.sampled, ax, name='Regularized', marker='*')
ax.legend()

Attention Maps
==============

In [ ]:
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable

def showAttention(fig, ax, input_sentence, attnn, kldn, attnk, kldk):
    # Set up figure with colorbar
#     ax = fig.add_subplot()
    attentions = np.stack([attnn, kldn, attnk, kldk], axis=0)
    mask1 = np.zeros(attentions.shape)
    mask1[1, :] = 1
    mask1[3, :]
    
    a1 = np.ma.masked_array(attentions, mask1)
    im = ax.imshow(a1, cmap='Reds', vmin=0, vmax=1, interpolation='none')
#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes("right", size="10%", pad=0.05)
#     fig.colorbar(im, cax=cax, ax=ax)
#     cbar = fig.colorbar(cax, orientation='vertical')
#     cbar.ax.set_yticklabels([])
    
    mask1 = np.zeros(attentions.shape)
    mask1[0, :] = 1
    mask1[2, :] = 1
    
    a1 = np.ma.masked_array(attentions, mask1)
    im = ax.imshow(a1, cmap='Greens', vmin=0, vmax=1, interpolation='none')
#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes("left", size="10%", pad=0.05)
#     fig.colorbar(im, cax=cax, ax=ax)
    #cbar.ax.set_yticklabels([])
    
    # Set up axes
    ax.set_xticks(np.arange(len(input_sentence)))
    ax.set_xticklabels(input_sentence, rotation=85)
    
    ax.set_yticks([])
    ax.set_yticklabels([])

#     plt.show()

In [ ]:
idxs = []
for idx, (att, sm) in enumerate(zip(model_normal.attn, model_normal.sampled)) :
    kld = (att * np.log((att + 1e-8)/(sm + 1e-8))).sum()
    if kld != kld :
        continue
    if kld > 1.5 :
        idxs.append(idx)

In [ ]:
not_eq = 0
for idx, (i, j) in enumerate(zip(normal_pred, kld_pred)) :
    if (i > 0.5) == (j > 0.5) :
        print(idx, i, j)
        not_eq += 1
print(not_eq)

In [ ]:
n = 422

In [ ]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("output_sst.pdf")
for i, n in enumerate(idxs) :
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))
    showAttention(fig, axes, [vec.idx2word[x] for x in Xtest[n]], model_normal.attn[n][:len(Xtest[n])], model_normal.sampled[n][:len(Xtest[n])],
                                                                model_kld.attn[n][:len(Xtest[n])], model_kld.sampled[n][:len(Xtest[n])])
    pdf.savefig(fig)
pdf.close()

In [ ]:
plt.show()

In [1]:
import torch

In [145]:
x = torch.randn(10, 30, 4, requires_grad=True)
W = torch.randn(4, requires_grad=True)
b = torch.randn(1, requires_grad=True)

f = torch.matmul(x, W) + b
a = torch.exp(f) / torch.exp(f).sum(-1).unsqueeze(-1)

h = torch.matmul(a.unsqueeze(1), x).squeeze()

W2 = torch.randn(4, 1, requires_grad=True)
b2 = torch.randn(1, requires_grad=True)
y = torch.matmul(h, W2) + b2

t = (torch.randn(10) > 0).float()
loss = torch.nn.BCEWithLogitsLoss()
loss = loss(y.squeeze(), t)
loss.backward()
b.grad.numpy()

array([7.683411e-09], dtype=float32)